# Hyperparameter optimisation
[Optuna docs](https://optuna.org)
[Aim docs](https://aimstack.readthedocs.io/en/stable/)

Let's have a look at hyperparameters optimisation (optuna) with experiment tracking (aim).

In [30]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

X, y = make_classification(n_samples=10000, n_informative=10, n_classes=3)

# It's a good pracrtice to leave chunk of test data which you use only once.
X_data, X_test, y_data, y_test = train_test_split(X, y, test_size=0.1) 

lb = LabelBinarizer().fit(y)

In [31]:
import optuna 
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from aim import Run


def get_accuracy(y, predictions):
    return accuracy_score(y, np.argmax(predictions, axis=1))

def get_auc(y, predictions):
    return roc_auc_score(lb.transform(y), predictions)
    
    
def objective(trial):
    trial_run = Run(experiment=f"lightgbm-{trial.number}")
    
    X_train, X_validation, y_train, y_validation = train_test_split(X_data, y_data, test_size=0.25)
    train = lgb.Dataset(X_train, label=y_train)
    validation = lgb.Dataset(X_validation, label=y_validation)
 
    param = {
            "num_class": 3,
            "objective": "multiclass",
            "metric": "auc_mu",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        }

    trial_run['hparams'] = param

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc_mu")
    gbm = lgb.train(
        param, train, valid_sets=[validation], callbacks=[pruning_callback]        
    )
    train_predictions = gbm.predict(X_train)
    validation_predictions = gbm.predict(X_validation)
    
    train_auc = get_auc(y_train, train_predictions)
    validation_auc = get_auc(y_validation, validation_predictions)
    
    train_accuracy = get_accuracy(y_train, train_predictions)
    validation_accuracy = get_accuracy(y_validation, validation_predictions)        
    
    trial_run.track(train_accuracy, name='accuracy', epoch=trial.number, context={ "subset":"train" })
    trial_run.track(validation_accuracy, name='accuracy', epoch=trial.number, context={ "subset":"validation" })
    trial_run.track(train_auc, name='auc', epoch=trial.number, context={ "subset":"train" })
    trial_run.track(validation_auc, name='auc', epoch=trial.number, context={ "subset":"validation" })
    return validation_auc
 
study = optuna.create_study(study_name='experiment',direction='maximize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-02-08 15:45:24,478] A new study created in memory with name: experiment
[I 2022-02-08 15:45:37,941] Trial 0 finished with value: 0.9819901656065678 and parameters: {'lambda_l1': 0.03236434327371788, 'lambda_l2': 1.3464980590539399e-08, 'num_leaves': 118, 'feature_fraction': 0.9033231317202044, 'bagging_fraction': 0.9556923677517071, 'bagging_freq': 5}. Best is trial 0 with value: 0.9819901656065678.
[I 2022-02-08 15:46:00,122] Trial 1 finished with value: 0.9787093131409725 and parameters: {'lambda_l1': 0.0016743798948805823, 'lambda_l2': 0.00036331606039578785, 'num_leaves': 246, 'feature_fraction': 0.8838231487536684, 'bagging_fraction': 0.9648706255903432, 'bagging_freq': 2}. Best is trial 0 with value: 0.9819901656065678.
[I 2022-02-08 15:46:16,782] Trial 2 finished with value: 0.9795117104061709 and parameters: {'lambda_l1': 1.2815210232012302e-05, 'lambda_l2': 0.0012098337176967384, 'num_leaves': 247, 'feature_fraction': 0.885706662506448, 'bagging_fraction': 0.6370061607

Number of finished trials: 10
Best trial: {'lambda_l1': 0.03236434327371788, 'lambda_l2': 1.3464980590539399e-08, 'num_leaves': 118, 'feature_fraction': 0.9033231317202044, 'bagging_fraction': 0.9556923677517071, 'bagging_freq': 5}


### Last evaluation on the data we put aside

In [32]:
from lightgbm.sklearn import LGBMClassifier
from goldilox import Pipeline

run = Run(experiment=f"lightgbm-final")
run['hparams'] = study.best_trial.params

model = LGBMClassifier(**study.best_trial.params).fit(X_data, y_data)

train_probabilities = model.predict_proba(X_data)
test_probabilities = model.predict_proba(X_test)

train_accuracy = get_accuracy(y_data, train_probabilities)
test_accuracy = get_accuracy(y_test, test_probabilities)
train_auc = get_auc(y_data, train_probabilities)
test_auc = get_auc(y_test, test_probabilities)

run.track(train_accuracy, name='accuracy', context={ "subset":"train" })
run.track(train_accuracy, name='accuracy', context={ "subset":"test" })
run.track(train_auc, name='auc', context={ "subset":"train" })
run.track(test_auc, name='auc', context={ "subset":"test" })

### For production we fit on the entire data
We save the metrices of the last final run and it's params to the pipeline so it is available.

In [33]:
from goldilox import Pipeline

pipeline = Pipeline.from_sklearn(LGBMClassifier(**study.best_trial.params),  
                                 variables={'hparams':study.best_trial.params,
                                            'metrices':{'train_accuracy':train_accuracy,
                                                   'test_accuracy':test_accuracy,
                                                   'train_auc':train_auc,
                                                   'test_auc':test_auc}}).fit(X,y)

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l1 is set=0.03236434327371788, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03236434327371788
[LightGBM] [Warning] feature_fraction is set=0.9033231317202044, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9033231317202044
[LightGBM] [Warning] bagging_fraction is set=0.9556923677517071, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9556923677517071
[LightGBM] [Warning] lambda_l2 is set=1.3464980590539399e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3464980590539399e-08


In [34]:
pipeline.inference(pipeline.raw)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,prediction
0,-1.104461,1.347733,1.131951,4.301946,-1.108296,0.763974,-4.688927,0.295072,-0.818672,0.95454,...,0.839913,-3.275066,3.49597,1.474378,-0.291933,-0.633007,-1.70554,-0.683422,1.861085,0


In [35]:
pipeline.variables

{'hparams': {'lambda_l1': 0.03236434327371788,
  'lambda_l2': 1.3464980590539399e-08,
  'num_leaves': 118,
  'feature_fraction': 0.9033231317202044,
  'bagging_fraction': 0.9556923677517071,
  'bagging_freq': 5},
 'metrices': {'train_accuracy': 1.0,
  'test_accuracy': 0.898,
  'train_auc': 1.0,
  'test_auc': 0.9759710820535412}}

### Let's look at the experiments

In [36]:
%load_ext aim
%aim up

The aim extension is already loaded. To reload it, use:
  %reload_ext aim
